# This version will try to learn what the most important words are

import files

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
import nltk
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize
from operator import itemgetter
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import ConfusionMatrixDisplay

load

In [2]:
df = pd.read_csv('../data/df.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

could do more here to explore the neither and both values

move on to language processing

train-test split

In [3]:
X, y = df['text'].to_frame(), df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

add label to X_train for research purposes .. obviously don't include this in the model

reset index to anticipate future problems ... or not reset the index???

In [4]:
X_train.loc[:, 'label'] = [y_train.loc[val] for val in X_train.index]

<ipython-input-4-0cb08c382a27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:, 'label'] = [y_train.loc[val] for val in X_train.index]


perfunctory exploring should happen here

top ten visualizations for pos. and non-pos.

size of vocabulary

more?

In [5]:
basic_token_pattern = r"(?u)\b\w\w+\b"
tokenizer = RegexpTokenizer(basic_token_pattern)

In [6]:
X_train.loc[:, 'text_tokenized'] = X_train['text'].apply(tokenizer.tokenize)
vocab_raw = set(X_train['text_tokenized'].explode())
print('Size of raw vocabulary:', len(vocab_raw))   

Size of raw vocabulary: 8876


<ipython-input-6-5ba9b3884ac5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:, 'text_tokenized'] = X_train['text'].apply(tokenizer.tokenize)


gonna need naive bayes, might not do any other models (markov, etc.)

In [7]:
baseline_model = MultinomialNB()

look at plurality winner to see score to beat

In [8]:
plurality_cv = round(y_train.value_counts(normalize=True)[0],4)
y_train.value_counts(normalize=True)

sentiment
0    0.672366
1    0.327634
Name: proportion, dtype: float64

first model, just ten features

In [9]:
tfidf = TfidfVectorizer(
    max_features = 10
)

X_train_vectorized = tfidf.fit_transform(X_train['text'])

baseline_cv = round(cross_val_score(baseline_model, X_train_vectorized, y_train).mean(),4)

print('Plurality:', plurality_cv,
      '\nBaseline: ',baseline_cv)

Plurality: 0.6724 
Baseline:  0.6724


an absolutely miniscule improvement

let's try all words, not just max_features = 10

In [10]:
tfidf = TfidfVectorizer()

X_train_vectorized = tfidf.fit_transform(X_train['text'])

all_words_cv = round(cross_val_score(baseline_model, X_train_vectorized, y_train).mean(),4)

print('Plurality:', plurality_cv,
      '\nBaseline: ', baseline_cv,
      '\nAll Words:', all_words_cv)

Plurality: 0.6724 
Baseline:  0.6724 
All Words: 0.7005


an actual improvement

let's look at which 10 terms were least and most associated with positive sentiment

In [11]:
# create the word list from this vectorizer with new index
feature_names = np.array(tfidf.get_feature_names())
# create array of the indices of the feature_names array, ordered by tfidf score
sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
# create frequency distribution (dictionary) of 1-grams
all_words_freq_dist = FreqDist(X_train['text_tokenized'].explode())

In [12]:
# show the words with the top 10 tfidf values, and their tfidf values
print('tfidf | frequency | word\n')
for n in range(-1,-21,-1):
    index = sorted_tfidf_index[n]
    print(
        round(X_train_vectorized.max(0).toarray()[0][index],4),
        all_words_freq_dist[feature_names[index]],
        feature_names[index]
         )      

tfidf | frequency | word

0.9622 1 worship
0.9278 10 rocks
0.9184 4 hmmmm
0.9014 1 covet
0.8927 2 orly
0.8842 106 location
0.8691 15 charging
0.8686 1 whoooooo
0.8622 5 applestore
0.8585 2 deleting
0.8434 24 atx
0.8336 41 oh
0.8262 13 circle
0.8253 1 craziness
0.8168 1 shice
0.8146 37 money
0.8111 2 denotes
0.8104 36 watch
0.8091 2382 rt
0.806 37 screen


In [13]:
# show the words with the bottom 10 tfidf values, and their tfidf values
print('tfidf | frequency | word\n')
for n in range(10):
    index = sorted_tfidf_index[n]
    print(
        round(X_train_vectorized.max(0).toarray()[0][index],4),
        all_words_freq_dist[feature_names[index]],
        feature_names[index]
         )      

tfidf | frequency | word

0.2333 1 á¾_î¾ð
0.2333 1 _ã
0.2333 1 _ô
0.2333 1 documents
0.2333 1 ¼¼
0.2333 1 çü
0.2333 1 öý
0.2333 1 èï
0.2341 7685 sxsw
0.2358 1 karaoke


experiment with min_df and max_df

In [14]:
tfidf = TfidfVectorizer(
    max_df = 5
)
X_train_vectorized = tfidf.fit_transform(X_train['text'])
# create the word list from this vectorizer with new index
feature_names = np.array(tfidf.get_feature_names())
# create array of the indices of the feature_names array, ordered by tfidf score
sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
# create frequency distribution (dictionary) of 1-grams
all_words_freq_dist = FreqDist(X_train['text_tokenized'].explode())

In [15]:
# show the words with the top 10 tfidf values, and their tfidf values
print('tfidf | frequency | doc frequency | word\n')
for n in range(-1,-21,-1):
    index = sorted_tfidf_index[n]
    print(
        round(X_train_vectorized.max(0).toarray()[0][index],4),
        all_words_freq_dist[feature_names[index]],
        len(X_train[X_train['text'].str.contains(feature_names[index])]),
        feature_names[index]
         )      

tfidf | frequency | doc frequency | word

1.0 5 5 crashes
1.0 3 3 mccannsxsw
1.0 1 1 recipient
1.0 2 2 recipes
1.0 3 3 huh
1.0 1 1 sxswbarcrawl
1.0 3 4 evil
1.0 5 5 sxswbuffalo
1.0 1 3 damm
1.0 3 3 mcommerce
1.0 2 2 sxswedu
1.0 1 2 blah
1.0 4 7 hrs
1.0 4 4 mcree
1.0 2 3 sxswgo
1.0 1 1 sxswgood
1.0 2 2 blast
1.0 1 1 sxswk
1.0 5 5 elusive
1.0 5 5 mechanics


In [16]:
# show the words with the bottom 10 tfidf values, and their tfidf values
print('tfidf | frequency | doc frequency | word\n')
for n in range(10):
    index = sorted_tfidf_index[n]
    print(
        round(X_train_vectorized.max(0).toarray()[0][index],4),
        all_words_freq_dist[feature_names[index]],
        len(X_train[X_train['text'].str.contains(feature_names[index])]),
        feature_names[index]
         )      

tfidf | frequency | doc frequency | word

0.2646 1 1 çü
0.2646 1 1 èï
0.2646 1 1 _ô
0.2646 1 1 _ã
0.2646 1 1 á¾_î¾ð
0.2646 1 1 öý
0.2646 1 1 documents
0.2646 1 1 ¼¼
0.2748 1 1 destroyed
0.2748 1 1 trajan


In [17]:
# show all instances of this word
X_train[X_train['text'].str.contains('worship')]
# the number of instances will often differ from the frequency
# because this will catch instances when this is juxtaposed with more characters
# e.g. 'covet' & 'coveting'

,text,label,text_tokenized
76,i worship @mention {link} #sxsw,0,"[worship, mention, link, sxsw]"


In [18]:
print('Smallest tfidf:\n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('Largest tfidf: \n{}'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

Smallest tfidf:
['çü' 'èï' '_ô' '_ã' 'á¾_î¾ð' 'öý' 'documents' '¼¼' 'destroyed' 'trajan']

Largest tfidf: 
['crashes' 'mccannsxsw' 'recipient' 'recipes' 'huh' 'sxswbarcrawl' 'evil'
 'sxswbuffalo' 'damm' 'mcommerce']


explore bigrams

let's try n-grams, n from 2 to 7, using all words

top ten only looks at top ten most frequent, so this is useless until you give it a stop words list, probably not useful at all

now i have a way to find the top and bottom tfidf scores, i need to take this one step further to see how frequent those terms are, this will help me to decide to ignore n > 4 for example

I will make a function to use with lambda to generate a column of ngrams

In [19]:
all_pos_words = X_train[X_train.label == 1]['text_tokenized'].explode()
all_pos_words_freq_dist = FreqDist(all_pos_words)

In [20]:
all_pos_words_set = set(all_pos_words)

In [21]:
all_pos_words_ordered = list(zip(*all_pos_words_freq_dist.most_common()))[0]

In [22]:
all_non_words = X_train[X_train.label == 0]['text_tokenized'].explode()
all_non_words_freq_dist = FreqDist(all_non_words)

In [23]:
all_non_words_set = set(all_non_words)

In [24]:
all_non_words_ordered = list(zip(*all_non_words_freq_dist.most_common()))[0]

In [25]:
m_pos = np.mean(list(all_pos_words_freq_dist.values()))
s_pos = np.std(list(all_pos_words_freq_dist.values()))

In [26]:
all_pos_words_freq_dist_normalized = {}

for key in all_pos_words_freq_dist:
    all_pos_words_freq_dist_normalized[key] = (all_pos_words_freq_dist[key] - m_pos) / s_pos
    

# all_pos_words_freq_dist_normalized = (all_pos_words_freq_dist - m) / s

In [27]:
m_non = np.mean(list(all_non_words_freq_dist.values()))
s_non = np.std(list(all_non_words_freq_dist.values()))

In [28]:
all_non_words_freq_dist_normalized = {}

for key in all_non_words_freq_dist:
    all_non_words_freq_dist_normalized[key] = (all_non_words_freq_dist[key] - m_non) / s_non

In [29]:
word_differential = {}
word_ratio = {}

for word in vocab_raw:
    a, b = 0, 0
    if word in all_pos_words_freq_dist_normalized:
        a = all_pos_words_freq_dist_normalized[word]
    if word in all_non_words_freq_dist_normalized:
        b = all_non_words_freq_dist_normalized[word]
    word_differential[word] = a - b
    if a * b != 0:
        word_ratio[word] = a / b

In [30]:
keys = list(word_differential.keys())
values = list(word_differential.values())
sorted_value_index = np.argsort(values)
sorted_dict = {keys[i]: values[i] for i in sorted_value_index}

In [34]:
for word in list(sorted_dict)[-201:]:
    for i in range(len(feature_names)):
        if feature_names[i] == word:
            index = i
    print(
        round(X_train_vectorized.max(0).toarray()[0][index],4),
        all_words_freq_dist[feature_names[index]],
        feature_names[index]
         )

0.5918 1 xooms
0.4749 1 adpeopleproblems
1.0 1 lepanele
0.4218 1 moore
0.4511 1 lecture
0.4472 1 alacagimiz
1.0 1 omarg
0.3873 1 imthatgood
0.4736 1 huuray
0.5774 1 moleskin
0.5965 1 digits
0.3766 1 compute
0.4304 1 mis
0.3622 1 dailydeals
0.5918 1 popupstores
0.4643 1 spaces
0.7342 1 manufacturing
0.5918 1 ser
0.3402 1 equates
0.4472 1 whats
1.0 1 wnt
0.5774 1 instrumental
0.5774 1 ami
0.7071 1 venn
0.5111 1 coveting
0.5223 1 armed
0.4538 1 objects
0.5774 1 contextclues
0.4593 1 49n4m
0.3325 1 shindig
0.4673 1 zzzs
1.0 1 coudbeeasier
0.5774 1 30k
1.0 1 producs
0.5147 1 marks
0.5192 1 sizzle
0.5 1 gingerman
0.5286 1 ûólots
0.5093 1 sitter
0.5055 1 hits
0.5774 1 appropriately
0.3894 1 crisiscommons
0.4511 1 rose
0.4301 1 burlesque
1.0 1 appleapple
1.0 1 arboretum
0.4112 1 snubor
0.7071 1 iphoneapp
0.3552 1 divorces
0.7229 1 n14
1.0 1 event_iap5507
0.3938 1 sketchily
0.3949 1 timberlake
0.4725 1 activate
0.5223 1 bluefly
0.7071 1 disturbing
0.5 1 motley
0.5206 1 pieces
0.3627 1 manual
0.

In [32]:
list(sorted_dict)[:10]

['mention',
 'sxsw',
 'link',
 'google',
 'rt',
 'quot',
 'to',
 'at',
 'circles',
 'social']